# Upper Confidence Bound BO


In [ ]:
from xopt import Xopt

# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")

The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [ ]:
# Make a proper input file.
YAML = """
generator:
  name: upper_confidence_bound
  beta: 0.1

evaluator:
  function: xopt.resources.test_functions.sinusoid_1d.evaluate_sinusoid

vocs:
  variables:
    x1: [0, 6.28]
  objectives:
    y1: 'MINIMIZE'
"""

In [ ]:
X = Xopt.from_yaml(YAML)
X

## Run Optimization

In [ ]:
X.random_evaluate(3)
for i in range(5):
    print(i)
    X.step()

## View output data

In [ ]:
X.data

## Visualize model

In [ ]:
fig, ax = X.generator.visualize_model(n_grid=100)